In [2]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import csv

# Initialize the data
def initData(filename, training_ratio, validation_ratio, test_ratio, attributes):
    with open(filename, 'r', newline='') as file:
        file_csv = csv.reader(file)
        full_header = next(file_csv)  # Read the full header row

        # Identify indices for the chosen attributes
        chosen_indices = [full_header.index(attr) for attr in attributes]
        labels = []
        data = []

        for row in file_csv:
            if len(row) < len(full_header):
                continue
            
            # Extract chosen attributes and convert to float
            features = [float(row[i]) for i in chosen_indices]
            
            # Assume the last column is the label
            label = row[-1]
            data.append(features + [label])
            
            if label not in labels:
                labels.append(label)

        len_data = len(data)
        print(f"Total rows read: {len_data}")

        # Split into training, validation, and test sets
        train_idx = int(training_ratio * len_data)
        val_idx = int((training_ratio + validation_ratio) * len_data)
        
        training_data = data[:train_idx]
        validation_data = data[train_idx:val_idx]
        test_data = data[val_idx:]

        if len(training_data) == 0 or len(validation_data) == 0:
            print("[ERROR] Training or validation set is empty. Check your data split.")
            exit()
    
    chosen_header = [full_header[i] for i in chosen_indices]
    return chosen_header, training_data, validation_data, test_data, labels


# %%
def minkowski_distance(point_1, point_2, p):
    return sum(abs(a - b)**p for a, b in zip(point_1, point_2))**(1/p)


# %%
def knn(training_data, new_point, k, p):
    distances = [
        (minkowski_distance(new_point, row[:-1], p), row[-1])
        for row in training_data
    ]
    distances.sort(key=lambda x: x[0])
    
    # Perform voting among the k nearest neighbors
    votes = {}
    for i in range(k):
        label = distances[i][1]
        votes[label] = votes.get(label, 0) + 1
    
    return max(votes, key=votes.get)

# %%
def compute_accuracy(training_data, data, k, p):

    if not data:
        return 0
    correct = sum(
        knn(training_data, point[:-1], k, p) == point[-1]
        for point in data
    )
    return correct / len(data)

# %%
dataset_file = 'final_pollution_dataset.csv'
attributes = ['SO2', 'CO', 'Proximity_to_Industrial_Areas']  # Two selected attributes

# Split ratios
training_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2

# Load data
chosen_header, training_data, validation_data, test_data, labels = initData(
    filename=dataset_file,
    training_ratio=training_ratio,
    validation_ratio=validation_ratio,
    test_ratio=test_ratio,
    attributes=attributes
)


Total rows read: 5000


In [3]:
from sklearn.preprocessing import StandardScaler

def scale_data(training_data, validation_data, test_data):
    """
    Scale features using StandardScaler while preserving labels
    """
    # Extract features and labels
    train_features = np.array([row[:-1] for row in training_data])
    val_features = np.array([row[:-1] for row in validation_data])
    test_features = np.array([row[:-1] for row in test_data])
    
    # Get labels
    train_labels = [row[-1] for row in training_data]
    val_labels = [row[-1] for row in validation_data]
    test_labels = [row[-1] for row in test_data]
    
    # Scale features
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_features)
    val_scaled = scaler.transform(val_features)
    test_scaled = scaler.transform(test_features)
    
    # Combine scaled features with labels
    training_scaled = [list(features) + [label] for features, label in zip(train_scaled, train_labels)]
    validation_scaled = [list(features) + [label] for features, label in zip(val_scaled, val_labels)]
    test_scaled = [list(features) + [label] for features, label in zip(test_scaled, test_labels)]
    
    return training_scaled, validation_scaled, test_scaled

# Add this after data loading in main():
training_data, validation_data, test_data = scale_data(training_data, validation_data, test_data)

In [4]:

# 1. Setup arrays for k and p values
k_values = list(range(1, 20))  # k from 1 to 9
p_values = list(range(1, 10))   # p from 1 to 5

# 2. Initialize accuracy matrices
val_accuracies = np.zeros((len(k_values), len(p_values)))
test_accuracies = np.zeros((len(k_values), len(p_values)))

# 3. Compute accuracies
for i, k in enumerate(k_values):
    for j, p in enumerate(p_values):
        acc_val = compute_accuracy(training_data, validation_data, k, p)
        acc_test = compute_accuracy(training_data, test_data, k, p)
        val_accuracies[i][j] = acc_val
        test_accuracies[i][j] = acc_test
        print(f"k={k}, p={p}: val_acc={acc_val:.3f}, test_acc={acc_test:.3f}")

print("Validation Accuracies:")
print(val_accuracies)
print("Test Accuracies:")
print(test_accuracies)

k=1, p=1: val_acc=0.854, test_acc=0.864
k=1, p=2: val_acc=0.852, test_acc=0.870
k=1, p=3: val_acc=0.852, test_acc=0.869
k=1, p=4: val_acc=0.858, test_acc=0.868
k=1, p=5: val_acc=0.856, test_acc=0.867
k=1, p=6: val_acc=0.856, test_acc=0.868
k=1, p=7: val_acc=0.858, test_acc=0.867
k=1, p=8: val_acc=0.860, test_acc=0.868
k=1, p=9: val_acc=0.858, test_acc=0.869
k=2, p=1: val_acc=0.854, test_acc=0.864
k=2, p=2: val_acc=0.852, test_acc=0.870
k=2, p=3: val_acc=0.852, test_acc=0.869
k=2, p=4: val_acc=0.858, test_acc=0.868
k=2, p=5: val_acc=0.856, test_acc=0.867
k=2, p=6: val_acc=0.856, test_acc=0.868
k=2, p=7: val_acc=0.858, test_acc=0.867
k=2, p=8: val_acc=0.860, test_acc=0.868
k=2, p=9: val_acc=0.858, test_acc=0.869
k=3, p=1: val_acc=0.896, test_acc=0.885
k=3, p=2: val_acc=0.882, test_acc=0.880
k=3, p=3: val_acc=0.888, test_acc=0.876
k=3, p=4: val_acc=0.888, test_acc=0.874
k=3, p=5: val_acc=0.884, test_acc=0.878
k=3, p=6: val_acc=0.884, test_acc=0.878
k=3, p=7: val_acc=0.884, test_acc=0.879


In [5]:
plt.figure(figsize=(16, 6))

# Validation accuracy subplot
plt.subplot(121)
sns.heatmap(val_accuracies,
            annot=True,
            fmt='.3f',
            xticklabels=p_values,
            yticklabels=k_values,
            cmap='viridis')
plt.title('Validation Accuracy')
plt.xlabel('p value')
plt.ylabel('k value')

# Test accuracy subplot
plt.subplot(122)
sns.heatmap(test_accuracies,
            annot=True,
            fmt='.3f',
            xticklabels=p_values,
            yticklabels=k_values,
            cmap='viridis')
plt.title('Test Accuracy')
plt.xlabel('p value')
plt.ylabel('k value')

# Save and display
plt.tight_layout()
plt.savefig('k_p_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()